In [1]:
# TACC: install if necessary
try:
    import numba
except:
    ! pip3 install --user numba numpy==1.19.1 scipy==1.5.2 tqdm==4.48.2
    INSTALL_PATH='/home/jupyter/mydata/jupyter_packages/lib/python3.6/site-packages'
    import sys, os
    if os.access(INSTALL_PATH, os.R_OK) and INSTALL_PATH not in sys.path:
        sys.path.insert(0, INSTALL_PATH)

# generic
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (8,6)
matplotlib.rcParams['font.size'] = 16
matplotlib.rcParams['animation.embed_limit'] = 60
matplotlib.rcParams['xtick.top'] = True
matplotlib.rcParams['ytick.right'] = True
import math
import numpy
import scipy.integrate
import scipy.stats
import scipy.special
from matplotlib import animation

In [123]:
# range of x
RNG=(-20,20)
# number of grid points
PTS = 4001
grid = numpy.linspace(RNG[0],RNG[1],PTS)[:-1] # drop last point, which wraps to initial point
deltaxsq = ((RNG[1]-RNG[0])/(PTS-1))**2

# parameters of the barrier
# set the edges of the barrier
BARRIEREDGES = (-0.25, 0.25)
# set the height of the barrier
BARRIERHEIGHT = 4.8

# momentum, don't adjust
P=3
#wave packet
psi=numpy.csingle(scipy.stats.norm(scale=3,loc=-12).pdf(grid))*numpy.exp(2j*math.pi*P*grid)
#eliminate packet beyond barrier for initial state
psi[grid >= BARRIEREDGES[0]] = 0
# normalize psi
psi /= math.sqrt(sum(psi*psi.conj()).real*(RNG[1]-RNG[0])/(PTS-1))

V = numpy.zeros_like(grid)
V[(grid>BARRIEREDGES[0]) & (grid<BARRIEREDGES[1])]=BARRIERHEIGHT


def d2psi(psi):
    return numpy.diff(psi,n=2,prepend=psi[-1],append=psi[0])/deltaxsq

def dpsidt(t, psi):
    global V
    return -1.j*(2*math.pi*V*psi - d2psi(psi)/(2*2*math.pi))


TRNG=(0,8)
TIMES=numpy.linspace(TRNG[0], TRNG[1], 100)
rk=scipy.integrate.solve_ivp(dpsidt, TRNG, psi, t_eval=TIMES)
print("Done with integration")
fig, ax = plt.subplots(figsize=(12,6))
ax2=ax.twinx()
linem, = ax.plot([],[],label='$|\psi|^2$')
linev,=ax2.plot(grid, V, color='r')
textleft=ax.text(-17, 0.7, '', fontsize='x-large')
textright=ax.text(5, 0.7, '', fontsize='x-large')
plt.xlim(*RNG)
ax.set_ylim(-0.,0.8)
ax2.set_ylim(0,5)
plt.legend([linem,linev], ['$|\psi|^2$', '$V$'])
def plotter(data):
    global rk
    psii = rk.y[:,data]
    linem.set_data(grid, (psii*psii.conj()).real)
    textleft.set_text(f'Left of barrier:\n$\int |\psi(x)|^2 dx$ = {numpy.trapz(numpy.abs(rk.y[:,data][grid<BARRIEREDGES[0]])**2, grid[grid<BARRIEREDGES[0]]):.4}')
    textright.set_text(f'Right of barrier:\n$\int |\psi(x)|^2 dx$ = {numpy.trapz(numpy.abs(rk.y[:,data][grid>BARRIEREDGES[1]])**2, grid[grid>BARRIEREDGES[1]]):.4}')
    return [linem,textleft,textright]

print("Rendering...")
ani = animation.FuncAnimation(fig, plotter, len(TIMES))
from IPython.display import HTML
plt.close()
HTML(ani.to_jshtml())

Done with integration
Rendering...
